In [31]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tweepy
from config import *

In [32]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [7]:
# Twitter handles

# @BBCNorthAmerica
# @CBSNews
# @CNN
# @FoxNews
# @nytimes

In [46]:
# Define function to obtain tweets

#NOTE: Change range to 5

def tweeter(handle):

    all_tweets = []
    
    for x in range(2):
        public_tweets = api.user_timeline(handle, page=x)
        all_tweets.append(public_tweets)
        tweet_list = [item for sublist in all_tweets for item in sublist]
    return tweet_list;

In [47]:
# Define function to analyze tweets

def vader(tweet):
    
    target_string = tweet['text']
    compound_score = analyzer.polarity_scores(target_string)['compound']
    return compound_score;

In [43]:
target = '@CNN'
with open('test_data.json', 'w') as outfile:  
    json.dump(tweeter(target), outfile)
print('Done!')

Done!


In [55]:
status = api.rate_limit_status()
#print(json.dumps(status, sort_keys=True, indent=4))
print(status['resources']['statuses']['/statuses/user_timeline']['remaining'])

888


In [59]:
# Analyze tweets
targets = ['@nytimes','@CNN','@FoxNews']

news_scores = {}

for name in targets:
    tweet_sentiment = []
    for item in tweeter(name):
        tweet = {}

        tweet['time'] = item['created_at']
        tweet['score'] = vader(item)
        tweet_sentiment.append(tweet)
    news_scores[name] = tweet_sentiment

In [60]:
print(news_scores)

{'@nytimes': [{'time': 'Mon Nov 27 07:08:04 +0000 2017', 'score': -0.4588}, {'time': 'Mon Nov 27 06:53:08 +0000 2017', 'score': 0.0}, {'time': 'Mon Nov 27 06:37:04 +0000 2017', 'score': 0.0258}, {'time': 'Mon Nov 27 06:22:24 +0000 2017', 'score': -0.4767}, {'time': 'Mon Nov 27 06:09:45 +0000 2017', 'score': -0.128}, {'time': 'Mon Nov 27 05:50:34 +0000 2017', 'score': -0.6908}, {'time': 'Mon Nov 27 05:32:34 +0000 2017', 'score': 0.296}, {'time': 'Mon Nov 27 05:17:39 +0000 2017', 'score': 0.0}, {'time': 'Mon Nov 27 05:00:20 +0000 2017', 'score': -0.1027}, {'time': 'Mon Nov 27 04:32:11 +0000 2017', 'score': -0.296}, {'time': 'Mon Nov 27 04:13:11 +0000 2017', 'score': 0.0}, {'time': 'Mon Nov 27 03:58:56 +0000 2017', 'score': -0.6124}, {'time': 'Mon Nov 27 03:44:43 +0000 2017', 'score': -0.3182}, {'time': 'Mon Nov 27 03:28:10 +0000 2017', 'score': 0.6369}, {'time': 'Mon Nov 27 03:08:08 +0000 2017', 'score': 0.3252}, {'time': 'Mon Nov 27 02:49:12 +0000 2017', 'score': 0.0}, {'time': 'Mon Nov